In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        img_width, img_height = 28, 28
        out_nodes_cnt = 10
        layer_nodes_cnt = (img_width * img_height, 512, 512, out_nodes_cnt)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(layer_nodes_cnt[0], layer_nodes_cnt[1]),
            nn.ReLU(),
            nn.Linear(layer_nodes_cnt[1], layer_nodes_cnt[2]),
            nn.ReLU(),
            nn.Linear(layer_nodes_cnt[2], out_nodes_cnt),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred  = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.1766, 0.2175, 0.5005, 0.0000, 0.1728,
         0.0000, 0.3563, 0.0496, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2095, 0.1888],
        [0.0000, 0.0000, 0.0644, 0.0000, 0.1281, 0.2051, 0.3869, 0.1299, 0.1150,
         0.0000, 0.0436, 0.2756, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0446, 0.0251],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.2732, 0.3163, 0.5187, 0.2882, 0.2564,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0931, 0.2091]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.1766, 0.2175, 0.5005, 0.0000, 0.1728,
         0.0000, 0.3563, 0.0496, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2095, 0.1888],
        [0.0000, 0.0000, 0.0644, 0.0000, 0.1281, 0.2051, 0.3869, 0.1299, 0.1150,
         0.0000, 0.0436, 0.2756, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0446, 0.0251],
       

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [16]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0181,  0.0056, -0.0053,  ...,  0.0146, -0.0148, -0.0088],
        [ 0.0353, -0.0274,  0.0097,  ...,  0.0057, -0.0158,  0.0090]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0101,  0.0319], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0329,  0.0394,  0.0417,  ...,  0.0357, -0.0221,  0.0297],
        [-0.0272,  0.0182,  0.0282,  ...,  0.0002,  0.0208, -0.0171]],
       device='cuda:0', grad_fn=<Sl